## Задачи компьютерного зрения
<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-005.png" width="700">



Кроме классификации CV решает и другие задачи.

Сегментация, детектирование и многое другое

## Формат разметки COCO

Прежде чем говорить о способах решения задач компьютерного зрения разберемся с форматами входных данных.

#### COCO - Common Objects in COntext

Один из наиболее популярных датасатов содержащий данные для сегментации и детектирования.

Содержит: 
- Категории
- Маски
- Ограничивающие боксы (*bounding boxes*)
- Описания (*captions*)
- Ключевые точки (*keypoints*)
- И многое другое

In [ ]:
#Фиксируем random_seed
import torch
import random
import numpy as np
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

#Выставлям device для расчетов
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Загрузим датасет

In [ ]:
import requests, zipfile, io

r = requests.get(
    "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

Для работы с датасетом используется пакет `pycocotools`

[Подробнее о том как создать свой COCO датасет с нуля](https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch).

In [ ]:
from pycocotools.coco import COCO

coco = COCO("annotations/instances_val2017.json")

Разберемся с форматом на примере одной записи

In [ ]:
catIds = coco.getCatIds(catNms=["cat"])  # Получаем ID котиков
print('ID класса "кот" = %i' % catIds[0])

imgIds = coco.getImgIds(catIds=catIds)  # Фильтруем датасет по тегу
print("Всего изображений с котами: %i" % len(imgIds))

Разберем что у нас в метаданных

In [ ]:
img_list = coco.loadImgs(imgIds[0])  # Берем только первую картинку
img = img_list[0]
img

Посмотрим на изображение

In [ ]:
import skimage.io as io
import matplotlib.pyplot as plt

I = io.imread(img["coco_url"])
plt.axis("off")
plt.imshow(I)
plt.show()

Сконвертируем в PIL формат для удобства дальнейшей работы

In [ ]:
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt


def coco2pil(url):
    print(url)
    response = requests.get(url)
    return Image.open(BytesIO(response.content))


pil_img = coco2pil(img["coco_url"])
plt.imshow(pil_img)

#### Категории в COCO

Давайте посмотрим на примеры категорий в нашем датасете. Отобразим каждую 10ую категорию

In [ ]:
cats = coco.loadCats(coco.getCatIds())  # Грузим категории
num2cat = {}  # Создаем словарь для хранения
print("Категории COCO: ")
for cat in cats:
    num2cat[cat["id"]] = cat["name"]
    if cat["id"] in range(0, 80, 10):
        print(cat["id"], ":", cat["name"], end="   \n")

В датасете так же есть категория **0**. Ее используют для обозначения класса фона.

Есть так же суперкатегории

In [ ]:
print(cats[2])
print(cats[3])

nms = set([cat["supercategory"] for cat in cats])
print("COCO supercategories: \n{}".format("\n".join(nms)))

#### Вернемся к метаданным

Помимо метаданных нам доступна разметка ([подробнее о разметке](!https://cocodataset.org/#format-data)), давайте ее загрузим и отобразим

In [ ]:
annIds = coco.getAnnIds(imgIds=img["id"])
anns = coco.loadAnns(annIds)

plt.imshow(I)
plt.axis("off")
coco.showAnns(anns)

Давайте разберем из чего состоит разметка

In [ ]:
def dump_anns(anns):
    for i, a in enumerate(anns):
        print(f"#{i}")
        for k in a.keys():
            if k == "category_id" and num2cat.get(a[k], None):
                print(k, ": ", a[k], num2cat[a[k]])  # Show cat. name
            else:
                print(k, ": ", a[k])


dump_anns(anns)

Для объектов, которых слишком много существует отдельная метка `iscorwd`

In [ ]:
plt.rcParams["figure.figsize"] = (120, 60)

catIds = coco.getCatIds(catNms=["people"])
annIds = coco.getAnnIds(catIds=catIds, iscrowd=True)
anns = coco.loadAnns(annIds[0:1])

dump_anns(anns)
img = coco.loadImgs(anns[0]["image_id"])[0]
I = io.imread(img["coco_url"])
plt.figure(figsize=(10, 10))
plt.imshow(I)
coco.showAnns(anns)  # People in the stands
seg = anns[0]["segmentation"]
print("Counts", len(seg["counts"]))
print("Size", seg["size"])

Как получить маску в виде массива?

In [ ]:
import numpy as np

annIds = coco.getAnnIds(imgIds=[448263])
anns = coco.loadAnns(annIds)
msk = np.zeros(seg["size"])

fig, ax = plt.subplots(nrows=4, ncols=4, figsize=(10, 10))

i = 0
for row in range(4):
    for col in range(4):
        msk = coco.annToMask(anns[i])
        ax[row, col].imshow(msk)
        ax[row, col].set_title(num2cat[anns[i]["category_id"]])
        i += 1

А еще у нас есть bounding boxes

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

annIds = coco.getAnnIds(imgIds=[448263])
anns = coco.loadAnns(annIds)

RGB_img = cv2.cvtColor(I, cv2.COLOR_BGR2RGB)

for i in range(len(anns)):
    x, y, width, heigth = anns[i]["bbox"]
    x, y, width, heigth = int(x), int(y), int(width), int(heigth)
    if anns[i]["category_id"] == 1:
        color = (255, 255, 255)
    if anns[i]["category_id"] == 37:
        color = (255, 0, 0)
    if anns[i]["category_id"] == 40:
        color = (0, 255, 0)
    RGB_img = cv2.rectangle(RGB_img, (x, y), (x + width, y + heigth), color, 2)
cv2_imshow(RGB_img)

И еще куча всего

##### $\color{brown}{\text{Допольнительная информация}}$ 
#### Еще более глубокое понимание разметки

Что такое [run-length encoding - RLE](https://en.wikipedia.org/wiki/Run-length_encoding)?

[Видео-разбор](https://www.youtube.com/watch?v=h6s61a_pqfM)

## Семантическая сегментация (*Semantic segmentation*)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-007.png" width="900">

Постановка задачи:

Предсказать класс для каждого пикселя.

Входные данные маска: 

[ x,y - > class_num ] 

Выходные данные маска:

[ x,y - > class_num ] 


###Способы решения



#### **a) Наивный.**

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-012.png" width="700">


Скользящим окном пройтись по изображению и предсказать клас для каждого пикселя с учетом его соседей.


#### **б) Разумный**

Убрать линейный слой в конце сети. По признакам во всех каналах определять класс каждого пикселя.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-015.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L09_CNN_Architectures/img/L09_CNN_Architectures_15.png"  width="700">

В лекции №8 мы говорили о том что сверту 1x1 можно рассматривать как полносвязанный слой.


Именно так она и будет использоваться при сегментации.

Количество классов будет соответствовать числу каналов.


Проблемы:
- нужно большое рецептивное поле, следовательно много слоев ( L 3х3 conv -> 1+2L receptive field)
- очень медленно на полноразмерных картах активации

#### **в) Эффективный**


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-017.png" width="700">


Вернмся к традиционной структуре со сжатием пространственных размеров. А после нее добавим разжимающий блок. 

### Autoencoder

Такая архитектура довольно популярна и применяется не только для сегментации: 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-07.png" width="700">

- сглаживание шума;
- снижение размерности -> вектор признак
- генерация данных


Об этом будет целая отдельная лекция чуть позже


#### Разжимающий (upsample) блок

#### Изменение размеров изображений 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-08.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/bilinear.png" width="300">

Билинейная интерполяция рассматривает квадрат 2x2 известных пикселя, окружающих неизвестный. В качестве интерполированного значения используется взвешенное усреднение этих четырёх пикселей.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/bicubic.png" width="300">

Бикубическая интерполяция идёт на один шаг дальше билинейной, рассматривая массив из 4x4 окружающих пикселей — всего 16. Поскольку они находятся на разных расстояниях от неизвестного пикселя, ближайшие пиксели получают при расчёте больший вес.

#### Upsample в Pytorch
С картами признаков можно обращаться так же как и с пикселями. Для этого в Pytorch используется метод Upsample


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-11.png" width="700">

In [ ]:
#Фиксируем random_seed
import torch
import random
import numpy as np
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

#Выставлям device для расчетов
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch
from torch import nn
import torchvision.transforms.functional as TF
import requests
from io import BytesIO
import matplotlib.pyplot as plt
from PIL import Image


def coco2pil(url):
    print(url)
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

def upsample(pil, ax, mode="nearest"):
    tensor = TF.to_tensor(pil)
    upsampler = nn.Upsample(scale_factor=2, mode=mode)
    tensor_128 = upsampler(tensor.unsqueeze(0))
    im_128 = TF.to_pil_image(tensor_128.squeeze()).convert("RGB")
    ax.imshow(im_128)
    ax.set_title(mode)
    ax.set_xlim(0, 20 * 2)
    ax.set_ylim(20 * 2, 0)


pic = coco2pil("http://images.cocodataset.org/val2017/000000448263.jpg")
pil_64 = pic.resize((64, 64))

fig, ax = plt.subplots(ncols=4, figsize=(15, 5))
ax[0].imshow(pil_64)
ax[0].set_title("Resized image")
ax[0].set_xlim(0, 20)
ax[0].set_ylim(20, 0)


upsample(pil_64, mode="nearest", ax=ax[1])
upsample(pil_64, mode="bilinear", ax=ax[2])
upsample(pil_64, mode="bicubic", ax=ax[3])

Обратите внимание что размер изображения увеличился в 2 раза!

#### MaxUnpooling

Разница с предыдущим методом в том что индексы элементов запоминаются.


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-019.png" width="700">


Сохраняем индексы каждого max pooling слоя.

При повышении разрешения копируем значения из выхода max pooling слоя с учетом запомненных индексов

[Документация к MaxPool2d](
https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html)

[Документация к MaxUnpool2d](
https://pytorch.org/docs/stable/generated/torch.nn.MaxUnpool2d.html?highlight=unpooling)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-14.png" width="700">

In [ ]:
import torch
from torch import nn
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt


def tensor_show(tensor, title="", ax=ax):
    im = TF.to_pil_image(tensor.squeeze()).convert("RGB")
    ax.set_title(title + str(im.size))
    ax.imshow(im)


pool = nn.MaxPool2d(kernel_size=2, return_indices=True)  # False by default
unpool = nn.MaxUnpool2d(kernel_size=2)

pil = coco2pil("http://images.cocodataset.org/val2017/000000448263.jpg")

fig, ax = plt.subplots(ncols=5, figsize=(20, 5), sharex=True, sharey=True)

ax[0].set_title("original " + str(pil.size))
ax[0].imshow(pil)
tensor = TF.to_tensor(pil).unsqueeze(0)
print("Initial shape", tensor.shape)

# Downsample
tensor_half_res, indexes1 = pool(tensor)
print("Indexes shape", indexes1.shape)
tensor_show(tensor_half_res, "1/2 down", ax=ax[1])


tensor_q_res, indexes2 = pool(tensor_half_res)
tensor_show(tensor_q_res, "1/4 down", ax=ax[2])

# Upsample
tensor_half_res1 = unpool(tensor_q_res, indexes2)
tensor_show(tensor_half_res1, "1/2 up", ax=ax[3])

print(indexes1.shape)
tensor_recovered = unpool(tensor_half_res1, indexes1)
tensor_show(tensor_recovered, "full size up", ax=ax[4])

Зачем нужен pad?


In [ ]:
import torch
import torch.nn.functional as F
import seaborn as sns

fig, ax = plt.subplots(ncols=2, figsize=(10, 5), sharex=True, sharey=True)
array = torch.ones((24, 24), dtype=int)
sns.heatmap(array, annot=True, fmt="d", ax=ax[0], cbar=False, vmin=0, vmax=1)
print("Размеры массива:", array.size())

array_padded = F.pad(array, pad=[4, 4])
sns.heatmap(array_padded, annot=True, fmt="d", ax=ax[1], cbar=False)
print("Размеры массива с padding:", array.size())

#### Transpose convolution

Способы восстановления пространственных размерностей которые мы рассмотрели, не содержали обучаемых параметров.


Обычная свертка:

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-16.png" width="700">



Upsample/transpose convolution

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-17.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-15.png" width="700">

[Блог-пост про 2d свертки с помощью пеермножения матриц](https://medium.com/@_init_/an-illustrated-explanation-of-performing-2d-convolutions-using-matrix-multiplications-1e8de8cd2544)

[Документация к ConvTranspose2d](
https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html?highlight=transpose#convtranspose2d)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-19.png" width="700">

In [ ]:
import torch
from torch import nn
import seaborn as sns

input = torch.randn(1, 16, 16, 16)
downsample = nn.Conv2d(16, 16, 3, stride=2, padding=1)

with torch.no_grad():
    upsample = nn.ConvTranspose2d(16, 16, 3, stride=2, padding=1)
    h = downsample(input)
    print("Downsampled size", h.size())

    output = upsample(h, output_size=input.size())
    print("Upsampled size", output.size())

fig, ax = plt.subplots(ncols=3, figsize=(15, 5), sharex=True, sharey=True)
sns.heatmap(input[0, 0, :, :], ax=ax[0], cbar=False, vmin=-2, vmax=2)
ax[0].set_title("Input")
sns.heatmap(h[0, 0, :, :], ax=ax[1], cbar=False, vmin=-2, vmax=2)
ax[1].set_title("Downsampled")
sns.heatmap(output[0, 0, :, :], ax=ax[2], cbar=False, vmin=-2, vmax=2)
ax[2].set_title("Upsampled")

### U-Net: Convolutional Networks for Biomedical Image Segmentation

Популярная архитектура для сегментации. Изначально была предложена ([оригинальная статья](https://arxiv.org/abs/1505.04597)) для анализа  медицинских изображений.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-20.png" width="700">

[ссылка](https://arxiv.org/abs/1505.04597)

[Реализация на PyTorch](https://github.com/milesial/Pytorch-UNet)

[U-Net на PyTorch Hub](https://pytorch.org/hub/mateuszbuda_brain-segmentation-pytorch_unet/)

[Статья-разбор](https://towardsdatascience.com/unet-line-by-line-explanation-9b191c76baf5)

Обратите внимание на серые стрелки на схеме

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-21.png" width="700">

При конкатенации пространственные размеры должны совпадать.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-22.png" width="700">

После upsample блоков ReLU не используется.


## Мультиклассовая сегментация

Если требуется сегментировать несколько класов объектов сразу, то выходной слой нужно изменить соответствующим образом.

Разберем несколько архитектур для мультиклассовой сегментации

### 2014 FCN 

Fully Convolutional Network
для того что бы не было путаницы с Fully Connected Network
последние именуют MLP (Multi Layer Perceptron)

[Пример реализации 1](https://pytorch.org/hub/pytorch_vision_fcn_resnet101/)

[Пример реализации 2](https://pytorch.org/vision/stable/models.html#semantic-segmentation)

Предобученная модель была обучена на части датасета COCO train2017 (на 20 категориях, представленых так же в датасете  Pascal VOC). Использовались следующие классы:

`['__background__', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']`

**КАРТИНКУ ЗАГРУЗИТЬ НА СЕРВЕР**

<img src ="https://miro.medium.com/max/1016/1*NXNGhfSyzQcKzoOSt-Z0Ng.png" width="550">

[ссылка](https://towardsdatascience.com/review-fcn-semantic-segmentation-eb8c9b50d2d1)

Работает довольно просто. Берем любой *back-bone* (например `ResNet50` или `VGG16`) и прикручиваем слой `upsample` до нужных нам размеров в конце. Посмотрим как работает:

In [ ]:
#Фиксируем random_seed
import torch
import random
import numpy as np
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

#Выставлям device для расчетов
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch
from torch import nn
from torchvision import transforms
import torchvision.transforms.functional as TF
import requests
from io import BytesIO
import matplotlib.pyplot as plt
from PIL import Image


def coco2pil(url):
    print(url)
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

classes = ["__background__", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor",
           ]

fcn_model = torchvision.models.segmentation.fcn_resnet50(
    pretrained=True, num_classes=21
)

preprocess = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),  # ImageNet
    ]
)

pil_img = coco2pil("http://images.cocodataset.org/val2017/000000448263.jpg")
input_tensor = preprocess(pil_img)

with torch.no_grad():
    output = fcn_model(input_tensor.unsqueeze(0))  # ['out'][0]

Возвращаются 2 массива

* out - каждый пиксель отражает ненормированную вероятность, соответствующую предсказанию каждого класса.

* aux - содержит значения *auxillary loss* на пиксель. На инференсе output['aux'] бесполезный.

In [ ]:
print(output.keys())  # Ordered dictionary
print("out", output["out"].shape, "Batch, class_num, h, w")
print("aux", output["aux"].shape, "Batch, class_num, h, w")

output_predictions = output["out"][0].argmax(0)  # for first element of batch
print(output_predictions.shape)

fig = plt.figure(figsize=(10, 10))
plt.imshow(pil_img)

indexes = output_predictions
fig, ax = plt.subplots(nrows=4, ncols=5, figsize=(10, 10))

i = 0
for row in range(4):
    for col in range(5):
        mask = torch.zeros(indexes.shape)
        mask[indexes == i] = 255
        # if mask.max() > 0:
        ax[row, col].set_title(classes[i])
        ax[row, col].imshow(mask)
        i += 1

### 2018 DeepLabv3

[Encoder-Decoder with Atrous Separable Convolution for Semantic Image Segmentation](https://arxiv.org/abs/1802.02611v3)

[Реализация на PyTorch](https://pytorch.org/vision/stable/models.html#deeplabv3)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-24.png" width="700">

В DeepLab появились следубщие новшевства: слои `Spatial pyramid pooling` и `Atros (dilated) convilutions`

#### Spatial pyramid pooling (SPP) layer

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-23.png" width="700">

[ссылка](https://arxiv.org/abs/1406.4729)

**Spatial Pyramid Pooling (SPP)** - это *pooling* слой, который устраняет ограничение фиксированного размера сети, т.е. CNN не требует входного изображения фиксированного размера. В частности, мы добавляем слой SPP поверх последнего конволюционного слоя. 

Слой SPP объединяет признаки и генерирует выходные данные фиксированной длины, которые затем поступают в MLP (или другие классификаторы). Другими словами, мы выполняем некоторую агрегацию информации на более глубоком этапе иерархии сети (между сверточными слоями и полностью связанными слоями), чтобы избежать необходимости обрезать или деформировать изображение в начале.

В pytorch нет отдельного модуля, но результат может быть получени применением нескольких AdaptiveMaxPool2d с разными размерами выходов.

#### Atros (Dilated) Convolution

**Dilated convolution** (расширенные свертки) - это тип свертки, который "раздувает" ядро, вставляя отверстия между элементами ядра. Дополнительный параметр (скорость расширения) указывает, насколько сильно расширяется ядро.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-25.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-26.png" width="700">

In [ ]:
from torch import nn
import torch
import seaborn as sns

# Atros example
with torch.no_grad():
    input = torch.tensor([[[1, 1, 1], [1, 1, 1], [1, 1, 1]]], dtype=torch.float)
    print("Input shape:", input.size())

    conv = nn.Conv2d(1, 1, kernel_size=2, dilation=1, bias=False)

    conv.weight = nn.Parameter(torch.tensor([[[[2, 2], [2, 2]]]], dtype=torch.float))

    out = conv(input.unsqueeze(0))
    print("Output:", out)

fig, ax = plt.subplots(ncols=3, figsize=(15, 5), sharex=False, sharey=False)
sns.heatmap(
    input[0], ax=ax[0], annot=True, fmt=".0f", cbar=False, vmin=0, vmax=8, linewidths=1
)
sns.heatmap(
    conv.weight.detach()[0, 0, :, :],
    ax=ax[1],
    annot=True,
    fmt=".0f",
    cbar=False,
    vmin=0,
    vmax=8,
    linewidths=1,
)
sns.heatmap(
    out[0, 0, :, :],
    ax=ax[2],
    annot=True,
    fmt=".0f",
    cbar=False,
    vmin=0,
    vmax=8,
    linewidths=1,
)

ax[0].set_title("Input")
ax[1].set_title("Kernel")
ax[2].set_title("Output")
fig.suptitle("Dilation = 1")

In [ ]:
conv = nn.Conv2d(
    1, 1, kernel_size=2, dilation=2, bias=False
)  # Fell free to change dilation

conv.weight = nn.Parameter(torch.tensor([[[[2, 2], [2, 2]]]], dtype=torch.float))

out = conv(input.unsqueeze(0))
print(out)
print(out.shape)

fig, ax = plt.subplots(ncols=3, figsize=(15, 5), sharex=False, sharey=False)
sns.heatmap(
    input[0], ax=ax[0], annot=True, fmt=".0f", cbar=False, vmin=0, vmax=8, linewidths=1
)
sns.heatmap(
    conv.weight.detach()[0, 0, :, :],
    ax=ax[1],
    annot=True,
    fmt=".0f",
    cbar=False,
    vmin=0,
    vmax=8,
    linewidths=1,
)
sns.heatmap(
    out[0, 0, :, :].detach(),
    ax=ax[2],
    annot=True,
    fmt=".0f",
    cbar=False,
    vmin=0,
    vmax=8,
    linewidths=1,
)

ax[0].set_title("Input")
ax[1].set_title("Kernel")
ax[2].set_title("Output")
fig.suptitle("Dilation = 2")

In [ ]:
input = torch.tensor([[[0, 1, 0], [1, 1, 1], [0, 1, 0]]], dtype=torch.float)
out = conv(input.unsqueeze(0))
print(out)

fig, ax = plt.subplots(ncols=3, figsize=(15, 5), sharex=False, sharey=False)
sns.heatmap(
    input[0], ax=ax[0], annot=True, fmt=".0f", cbar=False, vmin=0, vmax=8, linewidths=1
)
sns.heatmap(
    conv.weight.detach()[0, 0, :, :],
    ax=ax[1],
    annot=True,
    fmt=".0f",
    cbar=False,
    vmin=0,
    vmax=8,
    linewidths=1,
)
sns.heatmap(
    out[0, 0, :, :].detach(),
    ax=ax[2],
    annot=True,
    fmt=".0f",
    cbar=False,
    vmin=0,
    vmax=8,
    linewidths=1,
)

ax[0].set_title("Input")
ax[1].set_title("Kernel")
ax[2].set_title("Output")
fig.suptitle("Dilation = 2")

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-27.png" width="700">

##### $\color{brown}{\text{Допольнительная информация}}$ 

###### IoU - ценка точности

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-52.png" width="700">

[ссылка](http://datahacker.rs/deep-learning-intersection-over-union/)

###### Pixel-wise cross entropy loss


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-29.png" width="700">

In [ ]:
#Фиксируем random_seed
import torch
import random
import numpy as np
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

#Выставлям device для расчетов
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from torch import nn
import torch

one_class_out = torch.randn(1, 1, 32, 32)
one_class_target = torch.randn(1, 1, 32, 32)

# https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
cross_entropy = nn.CrossEntropyLoss()

# https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html
bce_loss = nn.BCEWithLogitsLoss()
loss = bce_loss(one_class_out, one_class_target)
print("BCE", loss)


two_class_out = torch.randn(1, 2, 32, 32)
two_class_target = torch.randint(1, (1, 32, 32))

print(two_class_out.shape)
print(two_class_target.shape)

loss = cross_entropy(two_class_out, two_class_target)

print("Cross entropy loss", loss)

###### DiceLoss

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/dice.jpeg" width="700">

[Блог-пост про семантическую сегментацию](https://www.jeremyjordan.me/semantic-segmentation/)

In [ ]:
from torch import nn


class BinaryDiceLoss(nn.Module):
    """Soft Dice loss of binary class
    Args:
        p: Denominator value: \sum{x^p} + \sum{y^p}, default: 2
        predict: A tensor of shape [N, *]
        target: A tensor of shape same with predict
       Returns:
        Loss tensor

    """

    def __init__(self, p=2, epsilon=1e-6):
        super().__init__()
        self.p = p  # pow degree
        self.epsilon = epsilon

    def forward(self, predict, target):
        predict = predict.flatten(1)
        target = target.flatten(1)

        # https://pytorch.org/docs/stable/generated/torch.mul.html
        num = torch.sum(torch.mul(predict, target), dim=1) + self.epsilon
        den = torch.sum(predict.pow(self.p) + target.pow(self.p), dim=1) + self.epsilon
        loss = 1 - 2 * num / den

        return loss.mean()  # over batch


criterion = BinaryDiceLoss()
output = torch.tensor([[[1, 1, 1], [1, 1, 1], [1, 1, 1]]], dtype=torch.float)


target = torch.tensor([[[1, 1, 1], [1, 1, 1], [1, 1, 1]]], dtype=torch.float)

soft_loss = criterion(output.unsqueeze(0), target.unsqueeze(0))
print("Loss", soft_loss)

# Детектирование (Object detection)


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-005.png" width="700">


- опредлить координаты прямоугольника в котором заключен объект.

Пример из задания для семинара:

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-05.png" width="700">


Задачу семантической сегментации мы решали через классификацию. Для детектирования разумно использовать регрессию.

В случае для одного объекта можно обучить модель предсказывать числа:

* координаты центра + ширину и высоту
* координаты правого верхнего и левого нижнего углов
* координаты вершин полигона ...



In [ ]:
#Фиксируем random_seed
import torch
import random
import numpy as np
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

#Выставлям device для расчетов
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from torchvision.models import resnet18
from torch import nn
import torch

# Возьмем обученную сеть
resnet_detector = resnet18(pretrained=True)

# Заменим голову на предсказание 4х точек (x1,y1 x2,y2)
resnet_detector.fc = nn.Linear(resnet_detector.fc.in_features, 4)  # x1,y1 x2,y2

criterion = nn.MSELoss()

# Это случайный пример. Не ожидайте результатов
input = torch.rand((1, 3, 224, 224))
target = torch.tensor([[0.1, 0.1, 0.5, 0.5]])  # x1,y1 x2,y2 or x,y w,h
output = resnet_detector(input)
loss = criterion(output, target)
print(output)
print("Loss:", loss)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-07.png" width="700">


[ссылка](https://arxiv.org/abs/2011.12619)

На прошлом семинаре мы упоминали про модели которые ищут ключевые точки на лице человека (MTCNN). Можно использовать тот же подход для поиска любых точек.


Начнем с ситуации когда объект один.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-08.png" width="500">

У нас есть предобученный классификатор.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-09.png" width="700">

и мы хотим научить его определять местоположение объекта.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-10.png" width="700">

Для этого можно использовать две лосс функции: одна - будет оценивать ошибку классификации, другая - локализации.

Результаты потребуется каким-то образом объединить. В простейшем случае - сложить.

##### $\color{brown}{\text{Допольнительная информация}}$ 

###### Regression loss

$\mathrm{MSE} = \frac{\sum^n_{i=1}(y_i-y_i^p)^2}{n}$ - L2/ MSE/ Mean Square Error/ Среднеквадратичная ошибка 


$\mathrm{MAE} = \frac{\sum^n_{i=1}|y_i-y_i^p|}{n}$ - L1/ MAE/ Mean Absolute Error/ Средняя ошибка


$\
    L_{\delta}(y, f(x))=\left\{
                \begin{array}{ll}
                  \frac{1}{2}(y-f(x))^2 \qquad \mathrm{for } |y-f(x)| \leq \delta\\
                  \delta|y-f(x)|-\frac{1}{2}\delta^2 \qquad \mathrm{otherwise}
                \end{array}
              \right.
  $ - Huber Loss/ Smooth Mean ABsolute Error/ Функция потерь Хьюбера


$L(y,y^p)=\sum_{i=1}^n log(cosh(y^p_i-y_i))$ - Log-Cosh Loss/ Логарифм гиперболического косинуса

 <img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-12.png" width="700">

[ссылка](https://heartbeat.fritz.ai/5-regression-loss-functions-all-machine-learners-should-know-4fb140e9d4b0)

MAE vs MSE

 <img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-13.png" width="700">

[ссылка](https://heartbeat.fritz.ai/5-regression-loss-functions-all-machine-learners-should-know-4fb140e9d4b0)

Huber vs Log-cos

###### Multitask loss

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-14.png" width="500">

[Multi-Task Learning Using Uncertainty to Weigh Losses for Scene Geometry and Semantics](https://arxiv.org/pdf/1705.07115.pdf)

[Пример реализации MultiTask learning](https://github.com/Hui-Li/multi-task-learning-example-PyTorch/blob/master/multi-task-learning-example-PyTorch.ipynb)


## Детектирование нескольких объектов

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-042.gif" width="700">



[ссылка](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture11.pdf)

### **a) Наивный способ решения: скользящее окно**

Перебрать все возможные местопоположения объектов и классифицировать фрагменты.


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-044.gif" width="700">


[ссылка](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture11.pdf)

- очень долго

### **b) Эвристика**

 Выбрать области в которых вероятность нахождения объекта наиболее высока (ROI = regions of interest) и проводить поиск только в них.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-049.png" width="700">

[ссылка](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture11.pdf)

### Selective search

Один из таких алгоритмов:

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-18.png" width="800">

[Статья про Selective Search](http://www.huppelen.nl/publications/selectiveSearchDraft.pdf)

Возвращает порядка 2000 прямоугольников для изображения.

С таким количеством уже можно работать.

[ссылка](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture11.pdf)

### R-CNN - Region CNN 
Построенна по такому принципу:

- на изображении ищутся ROI 
- для кажого делается resize 
- каждый ROI обрабатывается сверточной сетью
которая предсказывает класс кобъекта который в него попал



<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-055.png" width="700">

[ссылка](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture11.pdf)

Кроме класса модель предсказывает смещения для каждого bounding box

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-20.png" width="700">

[ссылка](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture11.pdf)

### NMS

Теперь возникает другая проблема в районе объекта алгоритм генерирует множество ограничивающих прямоугольников (bounding box) которые частично прекрывают друг друга.

Что бы избавиться от них используется другой алгоритм
Non maxima suppression

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-22.png" width="700">


Его задача избавиться об bbox которые накладиваются на истинный

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-23.png" width="700">

Для оценки схожести обычно используется метрика IoU
а заначение IoU при котором bbox считаются принадлежащими одному объекту является гиперпараметром (часто 0.5)

Soft NMS

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-24.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-25.png" width="700">

### Fast R-CNN

Проблеммой описанного выше подхода является скорость.
Так как мы вынужденны применять CNN порядка 2000 раз (в зависимости от эвристики которая генерирует ROI)


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-26.png" width="800">

[ссылка](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture11.pdf)

И решением является поиск ROI не на самом изображении, а на карте признаков, полученной после обработки всего изображения CNN. В таком случае большая часть сверток выполняется только один раз.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-27.png" width="700">

[ссылка](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture11.pdf)

Это радикально ускоряет процесс

### ROI Pooling

Появляется новая задача - 'resize' ROI на карте признаков.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-28.png" width="700">

[Документация Roi Pooling](https://pytorch.org/docs/stable/torchvision/ops.html#torchvision.ops.roi_pool)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-29.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-077.png" width="700">

[ссылка](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture11.pdf)

Скорость работы CNN снизилась и теперь узким местом становится эвристика для поиска ROI

### Faster R-CNN


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-078.png" width="700">

[ссылка](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture11.pdf)

**Идея: пусть сеть сама предсказывает ROI по карте признаков**

Для обучения требуется посчитать 4 loss.

### Region proposal network

Карта признаков имеет фиксированные и относительно небольшие пространственные размеры (например 20x15)


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-34.png" width="700">

Поэтому можно вернуться к идее скользящего окна которая была отвергнута в самом начале.

При этом можно использовать окна нескольких форм

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-35.png" width="700">

Предсказываются два значения:

* вероятность того что в ROI находится объект
* смещения

Сама сеть при этом может быть очень простой:

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/rpn.jpeg" width="700">

В результате скорость увеличивается почте в 10 раз

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-086.png" width="700">

[Модель на PyTorch](https://pytorch.org/vision/stable/models.html#faster-r-cnn)

In [ ]:
import torchvision

fr_rcnn = torchvision.models.detection.fasterrcnn_resnet50_fpn(
    pretrained=True, progress=True, num_classes=91, pretrained_backbone=True
)
fr_rcnn.eval()

Загрузим данные

In [ ]:
from pycocotools.coco import COCO
import requests
import zipfile
import io
import matplotlib.pyplot as plt

#Снова грузим данные
r = requests.get(
    "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()
coco = COCO("annotations/instances_val2017.json")


In [ ]:
from PIL import Image
from io import BytesIO

def coco2pil(url):
    print(url)
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

catIds = coco.getCatIds(catNms=["person", "bicycle"])
# person and bicycle
imgIds = coco.getImgIds(catIds=catIds)
img_list = coco.loadImgs(
    imgIds[12]
)  # http://images.cocodataset.org/val2017/000000370208.jpg
img = img_list[0]
print("Image data", img)

plt.figure(figsize=(10, 10))
pil_img = coco2pil(img["coco_url"])
plt.imshow(pil_img)

annIds = coco.getAnnIds(imgIds=img["id"])
anns = coco.loadAnns(annIds)
coco.showAnns(anns, draw_bbox=True)

In [ ]:
import torchvision.transforms.functional as TF
from PIL import ImageDraw
import matplotlib.pyplot as plt

with torch.no_grad():
    tensor = TF.pil_to_tensor(pil_img) / 255
    output = fr_rcnn(tensor.unsqueeze(0))
    draw = ImageDraw.Draw(pil_img)

    for i, bbox in enumerate(output[0]["boxes"]):
        if output[0]["scores"][i] > 0.5:
            draw.rectangle((tuple(bbox[:2].numpy()), tuple(bbox[2:].numpy())), width=2)
    plt.figure(figsize=(10, 10))
    plt.imshow(pil_img)
    plt.show()

### Two stage detector

Faster RCNN == Two stage detector

<img src ="http://edunet.kea.su/repo/src/
L12_Segmentation_Detection/img/lecture_12-089.png" width="700">

На среднем и верхнем слое выполняются очень похожие операции. Разница в том что на последнем слое предсказывается класс объекта, а промежуточном только вероятность его присутствия (objectness)

### One Stage detector

Если сразу предсказывать класс, то можно избавиться от второй стадии.

<img src ="https://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-1-1.png" width="700">

Детекторы работающие "за один проход":

YOLO, SSD, RetinaNet

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-1-2.png" width="700">

[Сравнение скорости моделей](https://pytorch.org/vision/stable/models.html#runtime-characteristics)

#### [SSD: Single Shot MultiBox Detector](https://arxiv.org/abs/1512.02325)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/ssd.png" width="700">

* модель VGG-16, предобученная на ImageNet
* manually defines a collection of aspect ratios to use for the B bounding boxes at each grid cell + offsets (x,y,w,h)
* напрямую предсказывает вероятность того, что класс присутствует в bounding box.
* есть класс для "background"

#### Retina Net - [Focal Loss for Dense Object Detection](https://arxiv.org/abs/1708.02002)



#### FocalLoss

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-43.png" width="1000">


[Блог-пост: Что такое Focal Loss и когда его использовать](https://amaarora.github.io/2020/06/29/FocalLoss.html)

#### Feature pyramyd network

[Feature Pyramid Networks for Object Detection](https://arxiv.org/pdf/1612.03144.pdf)

Это feature extractor для детекторов.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-21.png" width="700">

Так же как и в случае с сегментацией, точность повышается если делать предсказания на картах признаков разных масштабов.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/fpn1.jpeg" width="400">

Но первые слои содержат мало семантической информации (только низкоуровневые признаки). Из за этого детектирование(и сегментация) мелких объектов удается хуже.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/fpn2.png" width="400">

Идея состоит в том что бы делать предсказание с учетом семантической информации полученной на более глубоких слоях. 


При этом признаки суммируются.

In [ ]:
ВОТ ТУТ ПОКА ЗАКОНЧИЛА

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/fpn3.png" width="700">

Затем к новым картам признаков может применяться дополнительная свертка.


На выходе получаем карты признаков P2 - P5 на которых уже предсказываются bounding box.


В случае 2-stage детектора (RCNN) карты подаются на вход RPN 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/fpn5.jpeg" width="700">

А признаки для предсказаний используются из backbone 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/fpn6.jpeg" width="700">


RetinaNet использует выходы FPN и для предсказаний класса и bbox. 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-42.png" width="700">


[Блог-пост про FPN](https://jonathan-hui.medium.com/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c)



###YOLO

* [2016 You Only Look Once: Unified, Real-Time Object Detection.](https://arxiv.org/pdf/1506.02640.pdf) 
* [2017 YOLO9000: Better, Faster, Stronger.](https://arxiv.org/pdf/1506.02640.pdf)
* [2018 YOLOv3: An Incremental Improvement](https://arxiv.org/pdf/1804.02767.pdf)

* Апрель 2020. Alexey Bochkovskiy  “[YOLOv4: Optimal Speed and Accuracy of Object Detection"](https://arxiv.org/abs/2004.10934)
* Июнь 2020. [YOLOv5 Glenn Jocher](https://github.com/ultralytics/yolov5)
* Июль 2021. [YOLOX: Exceeding YOLO Series in 2021](https://arxiv.org/abs/2107.08430)

Первые версии проигрывали конкурентам. Но проект развивался. В настоящий момент это пожалуй оптимальный детектор по соотношению качество разпознавания/скорость.


##### $\color{brown}{\text{Допольнительная информация}}$ 
Старые версии YOLO

###### YOLOv3

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/yolov3.jpeg" width="700">

###### YOLOv4

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/yolov4.jpeg" width="700">
- SPP block
- Dense Block
- Больше слоев
- online Augmentation


#### YOLOv5


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-47.png" width="700">

Статья не публиковалась.
Точность сравнима  с v4 но модель определенно лучше упакованна.

In [ ]:
catIds = coco.getCatIds(catNms=["person", "bicycle"])
# person and bicycle
imgIds = coco.getImgIds(catIds=catIds)
img_list = coco.loadImgs(imgIds[5])
img = img_list[0]
print("Image data", img)

pil_img = coco2pil(img["coco_url"])
plt.figure(figsize=(10, 10))
plt.imshow(pil_img)

annIds = coco.getAnnIds(imgIds=img["id"])
anns = coco.loadAnns(annIds)
coco.showAnns(anns, draw_bbox=True)

Загрузка модели с Torch Hub



In [ ]:
import torch

# Load model from torch
model = torch.hub.load("ultralytics/yolov5", "yolov5s", pretrained=True)

Из коробки работает с изображениями в разных форматах и даже url, автоматически меняет размер входного изображения, возвращает объект с результатами ...

In [ ]:
from PIL import Image

# Apply yolov5 model
results = model(pil_img)
results.print()
results.save()  # image on disk

print(type(results.xyxy), len(results.xyxy), results.xyxy[0].shape)
results.pandas().xyxy[0]

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

cv_img = np.array(pil_img)
RGB_img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)

annos = results.pandas().xyxy[0]

for i in range(len(annos)):
    x_min, y_min, x_max, y_max = (
        int(annos["xmin"].iloc[i]),
        int(annos["ymin"].iloc[i]),
        int(annos["xmax"].iloc[i]),
        int(annos["ymax"].iloc[i]),
    )
    # width, height = int(annos['xmax']-annos['xmin']), int(annos['ymax']-annos['ymin'])
    if annos["name"].iloc[i] == "person":
        color = (255, 255, 255)
    if annos["name"].iloc[i] == "bicycle":
        color = (0, 0, 255)
    if annos["name"].iloc[i] == "backpack":
        color = (0, 255, 0)
    RGB_img = cv2.rectangle(RGB_img, (x_min, y_min), (x_max, y_max), color, 2)
cv2_imshow(RGB_img)

Однако если подать на вход модели тензор, выходы радикально меняются. YOLO переходит в режим обучения, что довольно не очевидно

In [ ]:
dummy_input = torch.rand((1, 3, 416, 416))
results = model(dummy_input)
print(type(results), len(results))
print(results[0].shape)
print(type(results[1]), len(results[1]))
for e in results[1]:
    print(e.shape)
# print(results[0], results[1].shape) # list of two elements

## Нard Example Mining

Представим что камера видеонаблюдения установленна на улице.
<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-48.png" width="700">

Целевые объекты, могут появляться достаточно редко (особено ночью)
Но на каждом из кадров будет фон, который будет сильно меняться в зависимости от освещения погодных условий и.т.п.

В результате возникнут изображения с образами которых не было в датасетах и они приведут к ложным срабатываниям.

Что бы дообучить сеть можно сохранять кадры с такими срабатываниями и добавлять их в датасет.



Online hard example mining

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-49.png" width="700">

Можно делать это непосредственно при обучении во время формирования batch-а

[Блог пост про Hard Mining Example](https://erogol.com/online-hard-example-mining-pytorch/)

Или даже заложить в структуру модели:

[Training Region-based Object Detectors with Online Hard Example Mining](https://arxiv.org/pdf/1604.03540.pdf)

[Loss Rank Mining: A General Hard ExampleMining Method for Real-time Detectors](https://arxiv.org/pdf/1804.04606.pdf)

# Instance Segmentation

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-005.png" width="700">


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-03.png" width="700">

[COCO panoptic](https://cocodataset.org/#panoptic-2020)

**Mask R-CNN** (Detectron) - концептуально простая, гибкая и общая схема сегментации объектов. Подход эффективно обнаруживает объекты на изображении и одновременно генерирует высококачественную маску сегментации для каждого объекта. 

Метод, названный Mask R-CNN, расширяет Faster R-CNN (который мы обсуждали ранее), добавляя ветвь для предсказания маски объекта параллельно с существующей ветвью для распознавания *bounding boxes*. 

Код доступен [тут](https://github.com/facebookresearch/Detectron)


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-096.png" width="700">







[Модель Mask R-CNN](https://pytorch.org/vision/stable/models.html#mask-r-cnn)

[Пример запуска Mask R-CNN есть в документации Pytorch](https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html)



### ROI Align


Выравнивание области интереса, или **RoI Align**, - это операция извлечения небольшой карты признаков из каждого RoI (Region of Interest) в задачах обнаружения и сегментации. Она устраняет жесткое квантование *RoI pool*, правильно выравнивая извлеченные признаки с входными данными. Чтобы избежать квантования границ или бинов RoI, RoIAlign использует билинейную интерполяцию для вычисления точных значений входных признаков в четырех регулярно дискретизированных местах в каждом бине RoI, а затем результат агрегируется (используя max или average).




<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-30.png" width="700">


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-31.png" width="700">

# Оценка качества детекции

## mAP - mean Average Precision

AP (*Average Precision* - средняя точность) - это популярная метрика для измерения качества детекторов объектов, таких как Faster R-CNN, SSD и др. Средняя точность вычисляет среднее значение *precision* (точности) для значения *recall* от 0 до 1. Звучит сложно, но на самом деле довольно просто. Давайте разберем на конкретных примерах. Но перед этим давайте вкратце разберем что такое *precision*, *recall* и IoU.

### Precision & recall

**Precision** измеряет, насколько точны ваши предсказания, т.е. процент правильных предсказаний.

**Recall** измеряет, насколько хорошо вы находите все положительные срабатывания (*positives*). Например, мы можем найти 80% возможных положительных срабатываний в наших K лучших предсказаниях.

Вот их математические определения:

$\mathrm{Precision} = \frac{TP}{TP+FP}$

$\mathrm{Recall} = \frac{TP}{TP+FN}$

где $TP$ - True Positive, $TN$ - True Negative, $FP$ - False Positive, $FN$ - False Negative.



Например, мы пытаемся детектировать яблоки на фотографиях. Предположим, мы обработали 20 фотографий (на 10 фотографиях по одному яблоку и на 10 фотографиях яблок нет) и обнаружили что:

* в 7 случаях наша нейросеть обнаружила яблоко там где оно было на самом деле (True Positive),
* в 3 случаях не обнаружила яблоко там где оно было (False Negative),
* в 4 случаях обнаружила яблоко там где его не было (False Positive)
* в 6 случаях правильно определила что на фотографии яблок нету (True Negative),



Посчитаем precision и recall:

In [ ]:
def precision(TP, FP):
    return TP/(TP+FP)

def recall(TP, FN):
    return TP/(TP+FN)

pres = precision(TP=7, FP=4)
rec  = recall(TP=7, FN=3)

print('Precision = %.2f' % pres)
print('Recall = %.2f' % rec)

### IoU (Intersection over union)

IoU измеряет перекрытие между двумя границами. Мы используем его для измерения того, насколько сильно наша предсказанная граница совпадает с истиной (границей реального объекта). В некоторых наборах данных мы заранее определяем порог IoU (например, 0.5) для классификации того, является ли предсказание True Positive или False Positive.

Например, рассмотрим предсказание сети для фотографии яблока:

In [ ]:
!wget https://st.depositphotos.com/1003272/1632/i/600/depositphotos_16322913-stock-photo-red-apple.jpg -O img.jpg

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

img = Image.open('img.jpg')
array = np.array(img)

fig,ax = plt.subplots(figsize=(5,5))

x0 = 75
y0 = 80
w0 = 450
h0 = 440

ground_truth = Rectangle((x0, y0), w0, h0, linewidth=2, edgecolor='g', linestyle='--', facecolor='none')

x1 = 90
y1 = 120
w1 = 500
h1 = 310
predicted = Rectangle((x1, y1), w1, h1, linewidth=2, edgecolor='b', facecolor='none')

ax.add_patch(ground_truth)
ax.add_patch(predicted)

ax.imshow(array)
ax.axis('off');

IoU опредлена как

$\mathrm{IoU} = \frac{\text{area of overlap}}{\text{area of union}}$

Посчитаем *area of overlap* и *area of union*:

In [ ]:
def area_of_overlap(x0, y0, x1, y1, w0, w1, h0, h1):
    x0_max = x0 + w0
    y0_max = y0 + h0
    x1_max = x1 + w1
    y1_max = y1 + h1
    dx = min(x0_max, x1_max) - max(x0, x1)
    dy = min(y0_max, y1_max) - max(y0, y1)
    if (dx>=0) and (dy>=0):
        return dx*dy

def area_of_rectangle(w, h):
    return w*h
# Найдем площадь пересечения предсказаний
a_of_overlap = area_of_overlap(x0, y0, x1, y1, w0, w1, h0, h1)

# Посчитаем индивидуальные площади прямоугольников
a_0 = area_of_rectangle(w0, h0)
a_1 = area_of_rectangle(w1, h1)

# Найдем площадь их union
a_of_union = a_0+a_1-a_of_overlap

print('Area of overlap = %i' % a_of_overlap)
print('Area of union = %i' % a_of_union)

Теперь посчитаем IoU

In [ ]:
IoU = a_of_overlap/a_of_union
print('IoU = %.2f' % IoU)

Посмотрим как будет меняться IoU в зависимости от качества предсказания

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets

def plot_predictions_and_calculate_IoU(x1, y1, w1, h1):
    fig,ax = plt.subplots(figsize=(5,5))

    x0 = 75
    y0 = 80
    w0 = 450
    h0 = 440

    ground_truth = Rectangle((x0, y0), w0, h0, linewidth=2, edgecolor='g', linestyle='--', facecolor='none')
    predicted = Rectangle((x1, y1), w1, h1, linewidth=2, edgecolor='b', facecolor='none')

    ax.add_patch(ground_truth)
    ax.add_patch(predicted)

    ax.imshow(array)
    ax.axis('off');

    a_of_overlap = area_of_overlap(x0, y0, x1, y1, w0, w1, h0, h1)
    a_0 = area_of_rectangle(w0, h0)
    a_1 = area_of_rectangle(w1, h1)
    a_of_union = a_0+a_1-a_of_overlap
    IoU = a_of_overlap/a_of_union
    print('IoU = %.2f' % IoU)

interact(plot_predictions_and_calculate_IoU, 
         x1 = widgets.IntSlider(min=0, max=array.shape[0], step=10, value=90), 
         y1 = widgets.IntSlider(min=0, max=array.shape[1], step=10, value=120), 
         w1 = widgets.IntSlider(min=0, max=array.shape[0], step=10, value=500),
         h1 = widgets.IntSlider(min=0, max=array.shape[1], step=10, value=310)
        );

Видим что чем лучше предсказание совпадает среальностью - тем выше у нас значение метрики IoU

### Average Precision

Продолжая тему яблок, возьмем все те же 20 фотографий на 10 из которых яблоки были, а на 10 яблок не было. И представим что у нас есть некая нейросеть, которая выдает следующие предсказания по всему датасету (или например по батчу):

In [ ]:
import pandas as pd

nn_preds = pd.DataFrame({'IoU' : [0.6, 0.98, 0.4, 0.3, 0.1, 0.96, 0.7, 0.3, 0.2, 0.8],  
                        'precision' : [1,1,0.67,0.5,0.4,0.5,0.57,0.5,0.44,0.5],
                        'recall' : [0.2,0.4,0.4,0.4,0.4,0.6,0.8,0.8,0.8,1]})
nn_preds


Будем считать что если $\mathrm{IoU} \geq 0.5$ - то предсказание правильное

In [ ]:
nn_preds['correct'] = False
nn_preds['correct'][nn_preds['IoU'] >= 0.5] = True

nn_preds

Можно заметить, что precision имеет зигзагообразный характер - она снижается при ложных срабатываниях и снова повышается при истинных срабатываниях. 

Давайте построим график precision от recall и убедимся

In [ ]:
fig,ax = plt.subplots(figsize=(10,3))
ax.plot(nn_preds.recall, nn_preds.precision)
ax.grid('on')
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')

По определению, что бы найти AP нужно найти площадь под кривой recall-precision:

$AP = \int_0^1p(r)dr$

Господи, интеграл! Какая жуть!

Precision и recall всегда находятся в пределах от 0 до 1. Поэтому AP также находится в пределах от 0 до 1. Перед расчетом AP для обнаружения объекта мы часто сначала сглаживаем зигзагообразный рисунок (на каждом уровне recall мы заменяем каждое значение precision максимальным значением точности справа от этого уровня отзыва)

In [ ]:
def smooth_precision(nn_preds):
    smooth_prec = []
    for i in range(len(nn_preds.precision.values)):
        max = nn_preds.precision.values[i:].max()
        smooth_prec.append(max)
    nn_preds['smooth_precision'] = smooth_prec
    return nn_preds

nn_preds = smooth_precision(nn_preds)

Давайте посмотрим как это выглядит на графике

In [ ]:
fig,ax = plt.subplots(figsize=(10,3))
ax.plot(nn_preds.recall, nn_preds.precision, label='precision', color='blue')
ax.plot(nn_preds.recall, nn_preds.smooth_precision, label='smooth precision', color='red')
ax.grid('on')
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.legend()

Зачем нам нужно сглаживание? Что бы снизить влияние случайных выбросов и "прыжков" в предсказаниях модели

Теперь давайте посчитаем таки AP

In [ ]:
fig,ax = plt.subplots(figsize=(10,3))
ax.plot(nn_preds.recall, nn_preds.precision, label='precision', color='blue')
ax.plot(nn_preds.recall, nn_preds.smooth_precision, label='smooth precision', color='red')
ax.fill_between(nn_preds.recall, nn_preds.smooth_precision, 
                np.zeros_like(nn_preds.smooth_precision), 
                color='red', alpha=0.1,
                label='Area under the curve')
ax.grid('on')
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_ylim(0.35,1.05)
ax.legend()

In [ ]:
from sklearn.metrics import auc

AP = auc(nn_preds.recall, nn_preds.smooth_precision)

print('AP = %.2f' % AP)

### COCO mAP

В последних исследовательских работах, как правило, приводятся результаты только для набора данных COCO. Для COCO AP - это среднее значение (*mean*) по нескольким IoU (минимальный IoU, который следует считать положительным совпадением). AP@[.5:.95] соответствует среднему AP для IoU от 0.5 до 0.95 с шагом 0.05. 

Давайте попробуем посчитать mAP. Для этого посчитает AP для каждого уровня IoU:

In [ ]:
import warnings
warnings.filterwarnings('ignore')

nn_prediction_at_iou = []
APs = []
for iou in np.arange(0.5,1,0.05):
    nn_preds_limited = nn_preds[nn_preds['IoU'] >= iou]
    nn_preds_limited = smooth_precision(nn_preds_limited)
    AP = auc(nn_preds_limited.recall, nn_preds_limited.smooth_precision)
    APs.append(AP)

In [ ]:
plt.plot(np.arange(0.5,1,0.05), APs, color='black')
plt.axhline(np.mean(APs), color='red', ls='--', label='mAP')
plt.xlabel('IoU')
plt.ylabel('AP')
plt.grid('on')
plt.legend();

print('mAP@[0.5:0.95] = %.2f' % np.mean(APs))

Есть несколько различных определений mAP, которые разняться от соревнования к соревнованию (суть одинковая, но разница в деталях подхода), поэтому для каждого соревнования лучше использовать их собственные библиотеки для расчета

##### $\color{brown}{\text{Допольнительная информация}}$ 
COCO mAP

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-51.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-52.png" width="700">

РАЗБИРАЕМ КАК СЧИТАЕТСЯ mAP

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/map_data.jpeg" width="700">



<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/gan/map.jpeg" width="1000">


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-54.png" width="700">




<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-56.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-57.png" width="1000">

[Конвертация результатов сегментации в COCO формат](https://www.javaer101.com/en/article/18652684.html).



In [ ]:
#Фиксируем random_seed
import torch
import random
import numpy as np
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

#Выставлям device для расчетов
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Синхронизируем метки Pascal2Coco

Грузим данные:

In [ ]:
import requests, zipfile, io

r = requests.get(
    "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [ ]:
from pycocotools.coco import COCO

pascal2coco = {}

classes = ["__background__", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor",
           ]


def find_in_dic(dic, val):
    for key in dic.keys():
        if dic.get(key, None) == val:
            return key
    return 0  # Assign missed classes to bg

coco = COCO("annotations/instances_val2017.json")
cats = coco.loadCats(coco.getCatIds())  # Грузим категории
num2cat = {}  # Создаем словарь для хранения
print("Категории COCO: ")
for cat in cats:
    num2cat[cat["id"]] = cat["name"]
    if cat["id"] in range(0, 80, 10):
        print(cat["id"], ":", cat["name"], end="   \n")

for i in range(1, len(classes)):  # Skip BG
    # if cats.get()
    coco_ind = find_in_dic(num2cat, classes[i])
    pascal2coco[i] = coco_ind

print(pascal2coco)

In [ ]:
import skimage.io as io
import matplotlib.pyplot as plt

img = coco.loadImgs(448263)[0]
print(img)
annIds = coco.getAnnIds(imgIds=[448263])
anns = coco.loadAnns(annIds)
I = io.imread(img["coco_url"])
plt.figure(figsize=(10, 10))
plt.imshow(I)

In [ ]:
from pycocotools import mask
import numpy as np
from itertools import groupby
import json
import torch
import torchvision
from torchvision import transforms
import requests
from io import BytesIO
import matplotlib.pyplot as plt
from PIL import Image

def coco2pil(url):
    print(url)
    response = requests.get(url)
    return Image.open(BytesIO(response.content))

def binary_mask_to_rle(binary_mask):
    rle = {"counts": [], "size": list(binary_mask.shape)}
    counts = rle.get("counts")
    for i, (value, elements) in enumerate(groupby(binary_mask.ravel(order="F"))):
        if i == 0 and value == 1:
            counts.append(0)
        counts.append(len(list(elements)))
    return rle

fcn_model = torchvision.models.segmentation.fcn_resnet50(
    pretrained=True, num_classes=21
)

preprocess = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),
    ]
)

pil_img = coco2pil("http://images.cocodataset.org/val2017/000000448263.jpg")
input_tensor = preprocess(pil_img)
                                             
with torch.no_grad():
    output = fcn_model(input_tensor.unsqueeze(0))  # ['out'][0]

indexes = output["out"][0].argmax(0)

detection_res = []
for i, cls_name in enumerate(classes):
    binary_mask = torch.zeros_like(indexes)
    binary_mask[indexes == i] = 1  # Form FCN for baseball

    if i > 0 and torch.max(binary_mask) > 0:
        uncompressed_rle = binary_mask_to_rle(binary_mask.numpy())  # encoded_gt,
        fortran_gt_binary_mask = np.asfortranarray(binary_mask).astype("uint8")
        encoded_gt = mask.encode(fortran_gt_binary_mask)
        bbox = list(mask.toBbox(encoded_gt))
        print(bbox)

        detection_res.append(
            {
                "score": 1.0,  # dummy
                "category_id": pascal2coco[i],
                "segmentation": uncompressed_rle,
                "bbox": bbox,
                "image_id": 448263,
                "iscrowd": 0,
            }
        )

print(detection_res)
with open("seg_res.json", "w", encoding="utf-8") as f:
    json.dump(detection_res, f, ensure_ascii=False, indent=4)

plt.subplot(1, 2, 1)
pil_img = coco2pil("http://images.cocodataset.org/val2017/000000448263.jpg")
plt.imshow(pil_img)
coco.showAnns(anns, draw_bbox=True)
plt.title("Annotation from COCO")

plt.subplot(1, 2, 2)
plt.imshow(pil_img)
coco.showAnns(detection_res, draw_bbox=True)
plt.title("Detection")

В предыдущем шаге мы посчитали предсказаниее от YOLO. Давайте оценим его точность

In [ ]:
import json

with open("seg_gt.json", "w", encoding="utf-8") as f:
    json.dump(anns, f, ensure_ascii=False, indent=4)

In [ ]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

# loadRes will generate a new COCO type instance based on coco_gt and return
coco_dt = coco.loadRes("seg_res.json")
coco_gt = coco.loadRes("seg_gt.json")

cocoEval = COCOeval(coco_gt, coco_dt, "bbox")  # 'segm', 'bbox'
# cocoEval.params.useSegm = True
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

print(cocoEval.stats)

# DINO - Self-supervised representation learning (with segmentation capabilities)
[Emerging Properties in Self-Supervised Vision Transformers](https://arxiv.org/abs/2104.14294)

[Отличное видео объяснение статьи](https://www.youtube.com/watch?v=h3ij3F3cPIk)

## Подготовка к запуску

In [ ]:
# ВНИМАНИЕ: ПОСЛЕ ЗАПУСКА ЭТОЙ ЯЧЕЙКИ ТРЕБУЕТСЯ ПЕРЕЗАПУСТИТЬ СРЕДУ ВЫПОЛНЕНИЯ 
# (Среда выполнения -> перезапустить среду выполнения)
!pip install -U augly
!sudo apt-get install python3-magic

Для начала подгрузим модель DINO (self-**DI**stillation with **NO** labels)

In [ ]:
#Фиксируем random_seed
import torch
import random
import numpy as np
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

#Выставлям device для расчетов
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
!git clone https://github.com/facebookresearch/dino.git

Теперь загрузим случайную картинку (можно выбрать любую, просто замените ссылку на свою)

In [ ]:
# URL = 'https://www.abenteuer-regenwald.de/uploads/media/content-l/01/1271-Yellow-headed_caracara_%28Milvago_chimachima%29_on_capybara_%28Hydrochoeris_hydrochaeris%29.jpg?v=1-0'
URL = 'https://limanpravda.com/files/uploads/2020/06/Samyj-smeshnoj-gryzun-kapibara.jpg'
!wget $URL -O test.jpg

## Результаты DINO

И посмотрим что с ней может сделать DINO, а затем обсудим как это работает и что вообще происходит

In [ ]:
!python /content/dino/visualize_attention.py --image_path /content/test.jpg

Пока не вдаваясь в детали, посмотрим на картинки которые генерирует DINO

In [ ]:
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols
    fig,ax = plt.subplots(nrows=rows, ncols=cols, figsize=(28,8))
    for num, img in enumerate(imgs):
        img_PIL = Image.open(img)
        ax[num].imshow(img_PIL)
        ax[num].set_xticks([])
        ax[num].set_yticks([])
    plt.subplots_adjust(hspace=0, wspace=0)

image_grid(imgs=sorted(glob('*.png'))[::-1], rows=1, cols=7)

## Как работает

Что бы понять что мы видим, давайте разберем архитектуру DINO и пойдем как ее обучали.
На самом деле, DINO не столько архитектура, сколько метод - то есть в качества *backbone* можно использовать любую нейросеть (например ResNet или ViT). Самые лучшие реузльтаты показали DINO на основе VIT, соответственно в этом блокноте будем разбирать именно эту конфигурацию.

**КАРТИНКУ ПОДГРУЗИТЬ НА СЕРВЕР**

<img src ="https://theaisummer.com/static/60c2cbe8edb845502c8bbc3b9e88791d/ee604/vision-transformer.png" width="700">

Давайте вспомним как работает ViT. **Vi**sual **T**ransformer получает на вход картинку разбитую на кусочки (*patches*) размеров 8 или 16 пикселей. Затем эти кусочки расправляются в вектор (*flatten*) и пропускаются через энкодер трансформера. Поверх трансформера прикручена MLP голова, которая собирает информацию с голов трансформера и предсказывает класс для картинки.



**КАРТИНКУ ПОДГРУЗИТЬ НА СЕРВЕР**

<img src ="https://neurohive.io/wp-content/uploads/2021/05/dino3-1-426x422.png" width="400">

DINO - это self-supervised метод, а значит классы ему недоступны. Что же делает эта сеть?

Разберем по шагам. DINO на вход получает изображение $x$.


In [ ]:
import os
import augly.image as imaugs
import augly.utils as utils
from IPython.display import display

input_img = imaugs.scale('test.jpg', factor=0.5)
display(input_img)

Дальше к этому изображению применяются две различные аугментации - $x_1$ и $x_2$

In [ ]:
transform_x1 = imaugs.Compose(
    [
     imaugs.Crop(0.1,0.1,0.6,0.6),
     imaugs.ColorJitter(
         brightness_factor=np.random.uniform(0.5,1),
         contrast_factor=np.random.uniform(0.5,1),
         saturation_factor=np.random.uniform(0.5,1)
     ),
     imaugs.RandomBlur(),
     imaugs.Resize(128,128)
     ]
    )

transform_x2 = imaugs.Compose(
    [
     imaugs.Crop(0.4,0.3,0.9,0.7),
     imaugs.ColorJitter(
         brightness_factor=np.random.uniform(0.5,1),
         contrast_factor=np.random.uniform(0.5,1),
         saturation_factor=np.random.uniform(0.5,1)
     ),
     imaugs.RandomBlur(max_radius=1),
     imaugs.Resize(128,128)
     ]
    )

aug_image_x1 = transform_x1(input_img)
aug_image_x2 = transform_x2(input_img)

fig,ax = plt.subplots(ncols=2)
ax[0].imshow(aug_image_x1)
ax[1].imshow(aug_image_x2)

Далее каждая из этих аугментация проходит через свою собственную версию **ViT** - *student* и *teacher*. Такой подход, когда с помощью большой сети учат сеть поменьше - называется *distillation*. В случае с DINO, они говорят о *self-distillation*, так как дистилляция происходит внутри одной сети.

В реальности, обучается только студент, а веса учителя обновляется как **E**xponential **M**oving **A**verage весов студента. Но пожалуй не будем залезать еще глубже.

In [ ]:
vits16 = torch.hub.load('facebookresearch/dino:main', 'dino_vits16') = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')

В конечном счете обе ветки сети выдают какое-то представление (*representation*) данных, которая, как мы надеемся будет близка для похожих изображений и далека для непохожих. Давайте на него посмотрим

In [ ]:
from torchvision import transforms

transform = transforms.ToTensor()

# Сконвертируем каждую аугментированную картинку в тензор
x1_tensor = transform(aug_image_x1).unsqueeze(0) 
x2_tensor = transform(aug_image_x2).unsqueeze(0)

# И прогоним через нашу обученную DINO
x1_representation = vits16(x1_tensor)
x2_representation = vits16(x2_tensor)

fig, ax = plt.subplots(ncols=2)
ax[0].imshow(x1_representation.view(24,16).detach().cpu().numpy())
ax[1].imshow(x2_representation.view(24,16).detach().cpu().numpy())
ax[0].set_title('$x_1$');
ax[1].set_title('$x_2$');

Выглядит и впрямь довольно похоже. А что если мы засунем туда что-то совершенно непохожее?

In [ ]:
? imaugs.RandomEmojiOverlay()

In [ ]:
transform_x3 = imaugs.Compose(
    [
     imaugs.Crop(0.0,0.0,0.2,0.2),
     imaugs.Resize(128,128),
     imaugs.RandomEmojiOverlay(emoji_size=0.4),
     imaugs.Resize(128,128),
     ]
    )

aug_image_x3 = transform_x3(input_img)

aug_image_x3

In [ ]:
x3_tensor = transform(aug_image_x3).unsqueeze(0) 

x3_representation = vits16(x3_tensor[:,:3,:,:])

fig, ax = plt.subplots(ncols=3)
ax[0].imshow(x1_representation.view(24,16).detach().cpu().numpy())
ax[1].imshow(x2_representation.view(24,16).detach().cpu().numpy())
ax[2].imshow(x3_representation.view(24,16).detach().cpu().numpy())
ax[0].set_title('$x_1$');
ax[1].set_title('$x_2$');
ax[2].set_title('$x_3$');

Да, выглядит не очень похоже

Loss для такой сети можно записать следующим образом: 

$\text{loss} = H(t_1, s_2)/2 + H(t_2, s_1)/2$,

где $ H(a, b) = −a \space log(b)$, $t_i$ - выученное представление i-ой аугментации teacher network и $s_i$ -  выученное представление i-ой аугментации student network

## Сегментация изображений

Давайте вновь посмотрим на результаты

In [ ]:
image_grid(imgs=sorted(glob('*.png'))[::-1], rows=1, cols=7)

Мы видим 6 карт внимания (*self-attention maps* - веса слоя self-attention) на 6 головах Visual Transformer. В результате self-supervised обучения по методике DINO, трансформер **САМОСТОЯТЕЛЬНО** придумал обращать внимание на различные части изображения, таким образом производя семантическую сегментацию

Мы можем все эти карты внимания объеденить в одно изображение, и просто назначить каждой карте свой цвет, а в качестве прозрачности использовать интенсивность

In [ ]:
import matplotlib.colors as mcolors
import numpy as np
from matplotlib import cm

def overlay(img, segmentations):
    img_PIL = Image.open(img)
    fig, ax = plt.subplots(ncols=2, figsize=(10,10))
    ax[0].imshow(img_PIL)
    ax[0].set_xticks([])
    ax[0].set_yticks([])
    
    ax[1].imshow(img_PIL.convert('LA'), alpha=0.5)
    for num, img in enumerate(segmentations):
        segment_PIL = Image.open(img).convert('LA')
        segment_arr = np.array(segment_PIL)
        colors = [(*cm.tab10(num)[:-1], c) for c in np.linspace(0,0.75,100)]
        cmap = mcolors.LinearSegmentedColormap.from_list('mycmap', colors, N=5)
        ax[1].imshow(segment_arr[:,:,0], cmap=cmap)
    ax[1].set_facecolor('black')
    ax[1].set_xticks([])
    ax[1].set_yticks([])
    plt.subplots_adjust(hspace=0, wspace=0)

overlay('/content/img.png', sorted(glob('*.png'))[:-1])

Видим, что DINO сегментирует разные части нашей картинки на разные семантические группы. В случае с капибарой - это голова, лицо (нос, глаза), животик и тело

## Сегментация видео

Но если вы думали что на этом все? То DINO может еще удивить. Например она умеет сегментировать видео

In [ ]:
!sudo pip install --upgrade youtube_dl

Скачаем видос (можно любой)

In [ ]:
URL = 'https://pixabay.com/videos/download/video-53486_large.mp4'
!youtube-dl -o video.mp4 $URL

И сгенерируем сегментированное видео

In [ ]:
!python /content/dino/video_generation.py --input_path /content/video.mp4 --output_path  /content/video_segmented --resize 360 640
!ffmpeg -i /content/video.mp4 -vf scale=640:360 /content/video_scaled.mp4
!ffmpeg \
  -i /content/video_scaled.mp4 \
  -i /content/video_segmented/video.mp4 \
  -filter_complex '[0:v]pad=iw*2:ih[int];[int][1:v]overlay=W/2:0[vid]' \
  -map '[vid]' \
  -c:v libx264 \
  -crf 23 \
  -preset veryfast \
  output.mp4

Посмотрим что получилось. Напоминаю, эта сеть вообще в режиме self-supervision училась!

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Кластеризация

**КАРТИНКУ ПОДГРУЗИТЬ НА СЕРВЕР**

А еще DINO умеет кластеризовать изображения. Выполнять не будем, так как процесс не быстрый, но можем посмотреть на результаты из их статьи

<img src ="https://scontent-vie1-1.xx.fbcdn.net/v/t39.2365-6/177871790_363624551852633_263144119734802610_n.gif?_nc_cat=105&ccb=1-3&_nc_sid=ad8a9d&_nc_ohc=aPyzhjD7npoAX_5yiBg&_nc_ht=scontent-vie1-1.xx&oh=7c93d5f7b5666f361571b278c68582d6&oe=611050F6" width="700">

# Список использованной литературы
[Подробнее о том как создать свой COCO датасет с нуля](https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch).

[Видео-разбор Run-Length Encoding](https://www.youtube.com/watch?v=h6s61a_pqfM)

[Блог-пост про 2d свертки с помощью пеермножения матриц](https://medium.com/@_init_/an-illustrated-explanation-of-performing-2d-convolutions-using-matrix-multiplications-1e8de8cd2544)

[Статья U-Net](https://arxiv.org/abs/1505.04597)
[Блог-пост про U-Net](https://towardsdatascience.com/unet-line-by-line-explanation-9b191c76baf5)

[Блог-пост про семантическую сегментацию](https://www.jeremyjordan.me/semantic-segmentation/)

[Multi-Task Learning Using Uncertainty to Weigh Losses for Scene Geometry and Semantics](https://arxiv.org/pdf/1705.07115.pdf)

[Статья про Selective Search](http://www.huppelen.nl/publications/selectiveSearchDraft.pdf)

[SSD: Single Shot MultiBox Detector](https://arxiv.org/abs/1512.02325)

[Focal Loss for Dense Object Detection](https://arxiv.org/abs/1708.02002)

[Блог-пост: Что такое Focal Loss и когда его использовать](https://amaarora.github.io/2020/06/29/FocalLoss.html)

[Feature Pyramid Networks for Object Detection](https://arxiv.org/pdf/1612.03144.pdf)

[2016 You Only Look Once: Unified, Real-Time Object Detection.](https://arxiv.org/pdf/1506.02640.pdf) 

[2017 YOLO9000: Better, Faster, Stronger.](https://arxiv.org/pdf/1506.02640.pdf)

[2018 YOLOv3: An Incremental Improvement](https://arxiv.org/pdf/1804.02767.pdf)

[YOLOv4: Optimal Speed and Accuracy of Object Detection](https://arxiv.org/abs/2004.10934)

[YOLOv5 Glenn Jocher](https://github.com/ultralytics/yolov5)

[Блог пост про Hard Mining Example](https://erogol.com/online-hard-example-mining-pytorch/)

[Training Region-based Object Detectors with Online Hard Example Mining](https://arxiv.org/pdf/1604.03540.pdf)

[Loss Rank Mining: A General Hard ExampleMining Method for Real-time Detectors](https://arxiv.org/pdf/1804.04606.pdf)